This is a notebook to generate all the prompts and seeds in the prompts google sheet

In [2]:
song_name = 'spacetrain' #@param {type:"string"}

import os
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

code_folder = '/content/gdrive/MyDrive/AI-Art Lee'
fp = os.path.join(code_folder, 'input_data.xlsx')

df_prompt = pd.read_excel(fp, 'prompts_{}'.format(song_name), index_col=0).dropna(how='all')
df_prompt

Mounted at /content/gdrive


,prompt,seeds,guidance_scale
name,,,
st_retroart,"a train travelling through space, stars, retro...","1251698433445435, 2287259771456094, 9195882365...",7.5
st_genre,"a train travelling through space, stars, Genre...","7513550102712487, 2019222925868596, 6484628732...",7.5
st_surreal,"a train travelling through space, stars, Surre...","6548786446951153, 6044285832786262, 8987207457...",7.5


In [3]:
%%capture
!pip install diffusers
!pip install transformers scipy ftfy accelerate

In [4]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4",
                                               torch_dtype=torch.float16,
                                               safety_checker=None
                                               )


pipe = pipe.to("cuda")



Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


# Iterate through prompts and seeds, outputting an image for both

In [12]:
for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(',')
  seeds = [s.strip() for s in seeds]
  print(seeds)

['1251698433445435', '2287259771456094', '919588236569255', '1345600503815601', '5232148067843148', '1780283600179318', '1590938731344866', '7514589968144458', '5953390400565685', '2805381572470346']
['7513550102712487', '2019222925868596', '6484628732178012', '5802207170251508', '8058345109409010', '6607177028541266', '8205029522770608', '3548472256980440', '2374342233491158', '6706329503192254']
['6548786446951153', '6044285832786262', '8987207457072248', '5588976376677084', '1535333058646368', '5556416826361356', '6466065110328318', '7939248234480387']


In [14]:
device = "cuda"
generator = torch.Generator(device=device)
width = 512
height = 512
skip_existing = True

output_folder = os.path.join(code_folder, song_name, 'prompt_images')

if not os.path.exists(output_folder): os.makedirs(output_folder)

for name, row in df_prompt.iterrows():
  seeds = row['seeds'].split(',')
  seeds = [s.strip() for s in seeds]

  prompt = row['prompt']
  guidance_scale = float(row['guidance_scale'])

  for seed in seeds:
    output_fn = "{}_{}.png".format(name, seed)

    if os.path.exists(os.path.join(output_folder, output_fn)):
      if skip_existing:
        print("{} already exists, skipping".format(output_fn))
        continue

    generator.manual_seed(int(seed))

    latent = torch.randn(
        (1, pipe.unet.in_channels, height // 8, width // 8),
        generator = generator,
        device = device
    )

    with torch.autocast(device):
      images = pipe(
          prompt,
          guidance_scale=guidance_scale,
          latents = latent,
      )

    output_image = images.images[0]

    output_image.save(os.path.join(output_folder, output_fn))

<ipython-input-14-4f5cb842dd25>:29: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  (1, pipe.unet.in_channels, height // 8, width // 8),


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]